# 5.3 延后初始化

到目前为止，我们忽略了建立网络时需要做的以下事情：  
  ·我们定义了网络架构，没有指定输入维度。  
  ·我们添加层时没有指定前一层的输出维度。  
  ·我们在初始化参数时，没有足够的信息来确定模型应该包含多少参数。  
 这里的诀窍是框架的延后初始化，即直到数据第一次通过模型传递时，框架才会动态地推断出每个层的大小。  

## 5.3.1 实例化网络

In [1]:
# 实例化一个多层感知机
import tensorflow as tf
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

D:\python39\python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


此时因为输入的维数是未知的，所以网络不可能知道输入层权重的维数。因此框架尚未初始化任何参数，我们通过访问以下参数进行确认。

In [2]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

请注意，每个层对象都存在，但权重为空。使用net.get_weight()将抛出一个错误，因为权重尚未初始化。  
接下来让我们将数据通过网络，最终使框架初始化参数。

In [3]:
X = tf.random.uniform((2, 20))
net(X)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]

一旦我们知道输入维数是20，框架就可以通过代植入20来识别一层的权重形状。识别出第一层的形状后，框架处理第二层，以此类推，直到所有的形状都已知为止。注意，在这种情况下，只有第一层你需要延迟初始化，但是框架仍是按顺序初始化的。等知道了所有的参数形状，框架就可以初始化参数。

## 5.3.2 小结

·延后初始化时框架能够自动推断参数形状，使修改模型框架变得容易，避免了一些常见的错误。  
·我们可以通过模型传递数据，使框架最终初始化参数。